In [3]:
import uvicorn
from fastapi import FastAPI

#fastapi인스턴스 생성
app = FastAPI()

#기본경로설정
@app.get("/")
def read_root():
    return{"message":"FastApi와 쥬피터 연결성공"}

#uvicorn을 백그라운드에서 실행(노트북 실행을 유지하면서 서버를 구동)
def run():
    uvicorn.run(app, host="0.0.0.0", port=8000)
    
import threading
server = threading.Thread(target=run)
server.start()

INFO:     Started server process [888]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
ERROR:    [Errno 10048] error while attempting to bind on address ('0.0.0.0', 8000): [winerror 10048] 각 소켓 주소(프로토콜/네트워크 주소/포트)는 하나만 사용할 수 있습니다
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.


In [4]:
import requests

response = requests.get("http://127.0.0.1:8000/")
print(response.json())

{'message': 'FastApi와 쥬피터 연결성공'}


In [2]:
mkdir templates

In [2]:
from diffusers import StableDiffusionPipeline
import torch
from fastapi import FastAPI, Request, Form
from fastapi.responses import HTMLResponse
from fastapi.templating import Jinja2Templates
from transformers import pipeline
from sqlalchemy import create_engine, Column, String, Integer
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
import threading
from PIL import Image
import numpy as np
import io
import base64

#데이터베이스설정
DATABASE_URL ="sqlite:///./test.db"
engine = create_engine(DATABASE_URL)
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)
Base = declarative_base()

#모델정의
class TextInput(Base):
    __tablename__="text_inputs"
    
    id = Column(Integer, primary_key=True, index=True)
    text =Column(String, index=True)
    
#데이터베이스 테이블생성 
Base.metadata.create_all(bind=engine)

#fastapi 앱 생성
app = FastAPI()

#jinja2템플릿 설정
templates = Jinja2Templates(directory="templates")

#Hugging Face 모델설정(감정분석 모델)
model = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")

device="cuda" if torch.cuda.is_available() else "cpu"
image_model = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4").to(device)

# image_model =pipeline("text-to-image", model="CompVis/stable-diffusion-v1-4")


#기본화면(프론트엔드)
@app.get("/", response_class=HTMLResponse)
def read_root(request: Request):
    return templates.TemplateResponse("index.html", {"request": request})

#사용자 입력을 처리하고 모델 결과 반환
@app.post("/predict/", response_class=HTMLResponse)
def predict(request: Request, text: str =Form(...)):
    #데이터베이스 세션 생성
    db = SessionLocal()
    
    #입력된 텍스트를 데이터베이스에 제공
    db_input =TextInput(text=text)
    db.add(db_input)
    db.commit()
    db.refresh(db_input)
    db.close()
    
    prediction = model(text)[0]
    
    #이미지 생성
    images = image_model(text).images
    
    #이미지처리(PIL이미지로 변환)
    image_bytes = io.BytesIO()
    # image = images[0]['image']
    images[0].save(image_bytes, format='PNG')
    image_bytes.seek(0)
    
    #이미지를 base64로인코딩
    image_base64 = base64.b64encode(image_bytes.getvalue()).decode('utf-8')
    
    return templates.TemplateResponse("result.html", {
        "request":request, 
        "text":text,
        "result": prediction,
        "image_data" : image_base64}
        ) #이미지데이터를 템플릿으로 전달

# 저장된 데이터 조회 엔드포인트
@app.get("/data/", response_class=HTMLResponse)
def get_data(request: Request):
    db = SessionLocal()
    texts = db.query(TextInput).all()  # 모든 텍스트 입력 조회
    db.close()
    
    return templates.TemplateResponse("data.html", {"request": request, "texts": texts})

#Uvicorn 서버 백그라운드에서 실행
def run():
    import uvicorn
    uvicorn.run(app,host="0.0.0.0", port=8000)
    
#백그라운드 스레드로 서버실행
server = threading.Thread(target=run)
server.start()

server.is_alive()

C:\Users\80412\AppData\Local\Temp\ipykernel_18216\298469755.py:20: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

True

INFO:     Started server process [18216]
INFO:     Waiting for application startup.
INFO:     Application startup complete.


ERROR:    [Errno 10048] error while attempting to bind on address ('0.0.0.0', 8000): [winerror 10048] 각 소켓 주소(프로토콜/네트워크 주소/포트)는 하나만 사용할 수 있습니다
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.


  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]